# **Data Mining for Confidence Level Detection**

Lydia Lonzarich and Katie Park
CPSC 322-01, Fall 2025

In [97]:
import importlib

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyRandomForestClassifier, MyDecisionTreeClassifier, MyNaiveBayesClassifier


import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

In [98]:
confidence_raw_data = MyPyTable().load_from_file("confidence_features.csv")
confidence = confidence_raw_data.new_deep_copy() # so the original dataset is not affected by normalization

attributes = ["body_lean_x", "shoulder_center_x", "hip_center_x", "spine_angle", "head_tilt_angle", "shoulder_slope", "head_direction", "arm_position", "posture"]
att_indices = []
for att in attributes: # finds the column indices of the attributes we're using
    att_indices.append(confidence.column_names.index(att))

confidence.normalize_columns([confidence.column_names[att] for att in att_indices if type(confidence.data[0][att]) != str])

# uses the discretizer 
for row_index, row in enumerate(confidence.data):
    for val_index, value in enumerate(row):
        if val_index in att_indices and type(confidence.data[row_index][val_index]) != str:
            confidence.data[row_index][val_index] = myutils.my_discretizer(confidence.data[row_index][val_index])



In [99]:
myTree = MyDecisionTreeClassifier()
myTree.fit(confidence.get_columns(attributes), confidence.get_column("confidence_label"))

In [100]:
myForest = MyRandomForestClassifier(N = 20, M = 7, F = 5)
myForest.fit(confidence.get_columns(attributes), confidence.get_column("confidence_label"))

y_pred = myForest.predict()

print(myevaluation.accuracy_score(myForest.y_test, y_pred))



KeyboardInterrupt: 

# **Introduction**

### **Dataset Used**

### **Findings**

In [3]:
# interview dataset
X_train_interview = [
        ["Senior", "Java", "no", "no"], # False
        ["Senior", "Java", "no", "yes"], # False
        ["Mid", "Python", "no", "no"], # True
        ["Junior", "Python", "no", "no"], # True 
        ["Junior", "R", "yes", "no"], # True
        ["Junior", "R", "yes", "yes"], # False 
        ["Mid", "R", "yes", "yes"], # True
        ["Senior", "Python", "no", "no"], # False 
        ["Senior", "R", "yes", "no"], # True 
        ["Junior", "Python", "yes", "no"], # True
        ["Senior", "Python", "yes", "yes"], # True 
        ["Mid", "Python", "no", "yes"], # True
        ["Mid", "Java", "yes", "no"], # True
        ["Junior", "Python", "no", "yes"] # False
    ]
y_train_interview = ["False", "False", "True", "True", "True", "False", "True", "False", "True", "True", "True", "True", "True", "False"]

test = MyRandomForestClassifier(N = 20, M = 7, F = 2)

test.fit(X_train_interview, y_train_interview)

current N: 20
['Attribute', 'att2', ['Value', 'no', ['Leaf', 'False', 5, 9]], ['Value', 'yes', ['Leaf', 'True', 4, 9]]]
current N: 20
['Attribute', 'att2', ['Value', 'no', ['Leaf', 'False', 5, 9]], ['Value', 'yes', ['Leaf', 'True', 4, 9]]]
current N: 20
['Attribute', 'att1', ['Value', 'Java', ['Leaf', 'False', 3, 9]], ['Value', 'Python', ['Leaf', 'True', 3, 9]], ['Value', 'R', ['Attribute', 'att0', ['Value', 'Junior', ['Leaf', 'False', 1, 3]], ['Value', 'Senior', ['Leaf', 'True', 2, 3]]]]]
current N: 20
['Attribute', 'att2', ['Value', 'no', ['Leaf', 'False', 5, 9]], ['Value', 'yes', ['Attribute', 'att0', ['Value', 'Junior', ['Leaf', 'False', 3, 4]], ['Value', 'Senior', ['Leaf', 'True', 1, 4]]]]]
current N: 20
['Attribute', 'att1', ['Value', 'Java', ['Leaf', 'False', 1, 9]], ['Value', 'Python', ['Leaf', 'True', 1, 9]], ['Value', 'R', ['Attribute', 'att0', ['Value', 'Junior', ['Leaf', 'False', 1, 7]], ['Value', 'Mid', ['Leaf', 'True', 4, 7]], ['Value', 'Senior', ['Leaf', 'True', 2, 7]]]]